In [1]:
import pandas as pd
from langdetect import detect, DetectorFactory
import os
import time
from tqdm import tqdm
from collections import Counter

# 📌 Configurar aleatoriedad para reproducibilidad
DetectorFactory.seed = 0

# 📌 Rutas del dataset
input_file = r"C:\Users\solan\Downloads\get_data_from_songs\data\df_lyrics_faltan_traduc_actualizado.csv"
output_file = r"C:\Users\solan\Downloads\get_data_from_songs\data\df_lyrics_faltan_traduc_idioma_faltaspoty.csv"
temp_file = r"C:\Users\solan\Downloads\get_data_from_songs\data\temp_language_detection.csv"

# 📌 Función para dividir letras largas en fragmentos
def split_text(text, chunk_size=500):
    """Divide un texto largo en fragmentos más pequeños para mejorar la detección de idioma."""
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

# 📌 Función mejorada para detectar idioma
def detect_language(text):
    try:
        if not isinstance(text, str) or len(text.strip()) < 10:  # Ignorar textos cortos
            return None
        
        # 📌 Si el texto es largo, dividir en fragmentos y detectar idioma en cada uno
        fragments = split_text(text, chunk_size=500)
        detected_languages = [detect(fragment) for fragment in fragments]
        
        # 📌 Determinar el idioma más frecuente
        most_common_lang = Counter(detected_languages).most_common(1)[0][0]
        return most_common_lang
    
    except Exception:
        return None  # Si hay error, devolver None en lugar de fallar

# 📌 Función para detectar idioma y guardar progreso
def add_language_column(input_file, output_file, save_interval=1000, pause_interval=5000):
    try:
        # 📌 Cargar dataset
        print("🔄 Cargando el dataset...")
        df = pd.read_csv(input_file, encoding="utf-8", low_memory=False)
        print("✅ Dataset cargado correctamente.")

        # 📌 Verificar si ya existe la columna 'language'
        if "language" not in df.columns:
            df["language"] = None

        # 📌 Cargar progreso si existe un archivo temporal
        if os.path.exists(temp_file):
            print("🔄 Archivo temporal encontrado. Cargando progreso...")
            df_temp = pd.read_csv(temp_file, encoding="utf-8")
            df.update(df_temp)  # Actualizar el dataset principal
            print("✅ Progreso recuperado.")

        # 📌 Filtrar filas sin idioma detectado
        missing_lang_indices = df[df["language"].isna() & df["lyrics"].notna()].index.tolist()
        print(f"🎯 Filas pendientes de procesar: {len(missing_lang_indices)}")

        if not missing_lang_indices:
            print("✅ No hay filas pendientes. Nada que procesar.")
            return

        # 📌 Procesar canciones sin idioma detectado con barra de progreso
        for i, idx in enumerate(tqdm(missing_lang_indices, desc="🔍 Detectando idioma", unit=" canción")):
            df.at[idx, "language"] = detect_language(df.at[idx, "lyrics"])

            # 📌 Guardar progreso cada `save_interval` filas
            if (i + 1) % save_interval == 0:
                df.to_csv(temp_file, index=False, encoding="utf-8")
                print(f"💾 Progreso guardado en {temp_file} ({i + 1} canciones procesadas).")

            # 📌 Pausar cada `pause_interval` filas para evitar sobrecarga
            if (i + 1) % pause_interval == 0:
                print(f"⏸️ Pausa breve tras {i + 1} filas procesadas para evitar sobrecarga...")
                time.sleep(10)  # Espera 10 segundos

        # 📌 Guardar archivo final
        df.to_csv(output_file, index=False, encoding="utf-8")
        print(f"✅ Archivo final guardado en {output_file}")

        # 📌 Eliminar el archivo temporal después de completar la detección
        if os.path.exists(temp_file):
            os.remove(temp_file)
            print("🗑️ Archivo temporal eliminado.")

    except FileNotFoundError:
        print(f"❌ El archivo {input_file} no se encontró. Verifica la ruta.")
    except Exception as e:
        print(f"⚠️ Se produjo un error inesperado: {e}")

# 📌 Ejecutar detección
if __name__ == "__main__":
    add_language_column(input_file, output_file, save_interval=1000, pause_interval=5000)


🔄 Cargando el dataset...
✅ Dataset cargado correctamente.
🎯 Filas pendientes de procesar: 20352


🔍 Detectando idioma:   5%|▍         | 1010/20352 [00:21<1:34:38,  3.41 canción/s]

💾 Progreso guardado en C:\Users\solan\Downloads\get_data_from_songs\data\temp_language_detection.csv (1000 canciones procesadas).


🔍 Detectando idioma:  10%|▉         | 2009/20352 [00:42<1:12:10,  4.24 canción/s]

💾 Progreso guardado en C:\Users\solan\Downloads\get_data_from_songs\data\temp_language_detection.csv (2000 canciones procesadas).


...

💾 Progreso guardado en C:\Users\solan\Downloads\get_data_from_songs\data\temp_language_detection.csv (19000 canciones procesadas).


🔍 Detectando idioma:  98%|█████████▊| 19993/20352 [06:53<00:02, 123.76 canción/s]

💾 Progreso guardado en C:\Users\solan\Downloads\get_data_from_songs\data\temp_language_detection.csv (20000 canciones procesadas).
⏸️ Pausa breve tras 20000 filas procesadas para evitar sobrecarga...


🔍 Detectando idioma: 100%|██████████| 20352/20352 [07:18<00:00, 46.36 canción/s] 


✅ Archivo final guardado en C:\Users\solan\Downloads\get_data_from_songs\data\df_lyrics_faltan_traduc_idioma_faltaspoty.csv
🗑️ Archivo temporal eliminado.


In [14]:
# valores unicos en la columna language
df_language = pd.read_csv(r"C:\Users\solan\Downloads\get_data_from_songs\data\df_lyrics_faltan_traduc_idioma_2_faltaspoty.csv")
df_language["language"].unique()

array(['en', 'it', 'es', 'ko', 'de', 'pt', 'fr', nan, 'tr', 'da', 'nl',
       'so', 'ja', 'ca', 'sv', 'fi', 'no', 'cy', 'lb', 'ru', 'et', 'af',
       'id', 'la', 'pl', 'sl', 'ga', 'ro', 'tl', 'zh', 'sk', 'lv', 'hi',
       'is', 'haw', 'ht', 'sw', 'hr', 'ig', 'xh', 'bs', 'hu', 'ha', 'el',
       'fy', 'fil', 'mi', 'fa', 'mt', 'ne', 'bg', 'co', 'mk', 'gd', 'sq',
       'cs', 'ar', 'ms', 'yo', 'ceb', 'vi', 'lt', 'ku', 'su', 'eo', 'st',
       'eu', 'ta', 'ny', 'jv', 'sn', 'gl', 'zu', 'mg', 'he', 'sd'],
      dtype=object)

In [16]:
# 📌 Contar filas con `❓` en `language`
missing_lang_rows = df_language [df_language ["language"] == "❓"]
print(f"Filas con '❓': {len(missing_lang_rows)}")

Filas con '❓': 0


In [21]:
# Análisis de valores nulos
missing_values = df_language.isnull().sum()
missing_percentage = (missing_values / len(df_language)) * 100

# Crear un resumen de los nulos
missing_summary = pd.DataFrame({
    'Column': df_language.columns,
    'Missing Values': missing_values,
    'Missing Percentage': missing_percentage
}).sort_values(by='Missing Percentage', ascending=False)

# Mostrar columnas con nulos
print(missing_summary[missing_summary['Missing Percentage'] > 0])


                                Column  Missing Values  Missing Percentage
translated_lyrics    translated_lyrics          142842           90.514603
playlists_names        playlists_names           63084           39.974400
positions                    positions           63084           39.974400
playlist_ids              playlist_ids           63084           39.974400
track_uri                    track_uri           63072           39.966796
views                            views           43204           27.377052
duration_ms                duration_ms            4451            2.820462
lyrics                          lyrics            2711            1.717878
language                      language            2705            1.714076
combined_genres        combined_genres            1174            0.743928
album_name                  album_name             480            0.304161
album_release_date  album_release_date             462            0.292755
popularity               

artist_name               0
song_name                 2
recording_id              0
danceable                 0
not_danceable             0
                      ...  
playlist_ids          63084
positions             63084
playlists_names       63084
combined_genres        1174
translated_lyrics    142842
Length: 88, dtype: int64

In [12]:
import pandas as pd
from langdetect import detect, DetectorFactory
from collections import Counter

# 📌 Configurar aleatoriedad para reproducibilidad
DetectorFactory.seed = 0

# 📌 Rutas del dataset
file_path = r"C:\Users\solan\Downloads\get_data_from_songs\data\df_lyrics_faltan_traduc_idioma_2_faltaspoty.csv"

# 📌 Cargar dataset
df = pd.read_csv(file_path, encoding="utf-8", low_memory=False)

# 📌 Función para dividir letras largas
def split_text(text, chunk_size=500):
    """Divide un texto largo en fragmentos más pequeños para mejorar la detección de idioma."""
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

# 📌 Función mejorada para detectar idioma
def detect_language(text):
    try:
        if not isinstance(text, str) or len(text.strip()) < 10:
            return None  # Ignorar textos cortos

        # 📌 Si el texto es largo, dividir en fragmentos y detectar idioma en cada uno
        fragments = split_text(text, chunk_size=500)
        detected_languages = [detect(fragment) for fragment in fragments]

        # 📌 Determinar el idioma más frecuente
        most_common_lang = Counter(detected_languages).most_common(1)[0][0]
        return most_common_lang

    except Exception:
        return None  # Si hay error, devolver None

# 📌 Filtrar solo las filas con `❓`
rows_with_unknown = df["language"] == "❓"
print(f"🔍 Filas a re-detectar: {rows_with_unknown.sum()}")

# 📌 Aplicar detección solo a las letras con `❓`
df.loc[rows_with_unknown, "language"] = df.loc[rows_with_unknown, "lyrics"].apply(detect_language)

# 📌 Reemplazar los `❓` restantes con `NaN`
df["language"].replace("❓", pd.NA, inplace=True)

# 📌 Guardar el archivo corregido
df.to_csv(file_path, index=False, encoding="utf-8")
print(f"✅ Archivo actualizado sin '❓' en `language`: {file_path}")


🔍 Filas a re-detectar: 2703


C:\Users\solan\AppData\Local\Temp\ipykernel_11924\3532347958.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["language"].replace("❓", pd.NA, inplace=True)


✅ Archivo actualizado sin '❓' en `language`: C:\Users\solan\Downloads\get_data_from_songs\data\df_lyrics_faltan_traduc_idioma_2_faltaspoty.csv
